# Ungraded Lab: Training with ImageDataGenerator


In [ ]:
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

In [4]:
import zipfile

#unzip the dataset
local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')
zip_ref.close()

In [5]:
#Defining the directories

import os

#Directory with our training horse pictures
train_horse_dir = os.path.join('./horse-or-human/horses')

#Directory with our training human pictures
train_human_dir = os.path.join('./horse-or-human/humans')

In [7]:
#We can see what the filenames look like in the horses and humans training directories

train_horses_names = os.listdir(train_horse_dir)
print(train_horses_names[:10])

train_human_names = os.listdir(train_human_dir)
print(train_human_names[:10])

['horse40-0.png', 'horse13-3.png', 'horse29-8.png', 'horse40-2.png', 'horse18-9.png', 'horse13-1.png', 'horse31-1.png', 'horse28-5.png', 'horse41-4.png', 'horse01-6.png']
['human06-11.png', 'human12-22.png', 'human14-29.png', 'human16-13.png', 'human13-23.png', 'human08-26.png', 'human10-27.png', 'human12-15.png', 'human01-28.png', 'human02-27.png']


#Building a Small Model from Scratch

In [8]:
import tensorflow as tf

In [9]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color

    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

In [11]:
from tensorflow.keras.optimizers import RMSprop

In [12]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer = RMSprop(learning_rate = 0.001),
    metrics = ['accuracy']
)

#Data Preprocessing

We read the pictures in the source folders, convert them to `float32` tensors, and feed them (with their labels) to the model.

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#All images rescaled by 1/255
train_datagen = ImageDataGenerator(rescale = 1/255)

#Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    './horse-or-human/',
    target_size = (300,300),
    batch_size = 128,
    class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.


#Training

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = 8,
    epochs = 15,
    verbose = 1
)

#Model Prediction
This code will allow you to choose 1 or more files from your file system, upload them, and run them through the model, giving an indication of whether the object is a horse or a human.

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():

  #predicting images
  path = '/content/' + fn
  img = load_img(path, target_size = (300,300))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis = 0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size = 10)
  print(classes[0])

  if classes[0]>0.5:
    print(fn + "is a human")
  else:
    print(fn + "is a horse")